In [63]:
using Pkg; Pkg.activate(".")
# Import libraries
using Flux, Plots, Random, Statistics, LaplaceRedux, MLUtils, JSON, Serialization

  Activating new project at `c:\Users\adeli\OneDrive\Desktop\facultate\2nd year\Q4 - Software Project\LaplaceRedux.jl\dev\notebooks\nn_encoding`


In [64]:
using LaplaceRedux.Data
n = 300       # number of observations
σtrue = 0.30  # true observational noise
x, y = Data.toy_data_regression(n; noise=σtrue)
xs = [[x] for x in x]
X = permutedims(x)

data = zip(xs, y)

print(data)

zip([[5.39660750802315], [0.8600796980177723], [0.700796111008966], [0.7871787269639672], [6.995967409927017], [2.8894065671261755], [4.082501846709352], [0.807311911539137], [0.5945226117480598], [3.0209824837043566

], [3.643622926819873], [6.44914835495354], [0.7233274629877888], [7.244234244904483], [6.448006884080103], [5.767595475697681], [5.277297808838903], [7.973130894167173], [2.2381911034465096], [2.2394568493732177], [4.936158039222256], [1.749680011152206], [2.267258247190596], [0.9201986947251077], [3.6885024480546837], [4.714522190836359], [5.617697608276963], [6.936777456718677], [0.6344271836805468], [0.16889674354277862], [3.6372002164462964], [0.7204118904924393], [3.289005310894492], [7.634299404268292], [7.30602907013772], [7.666371691859476], [4.091686440837934], [3.5651567326173295], [4.133295451053833], [3.8173789756045897], [1.0958458793651262], [0.3619128603934154], [3.2215124553833636], [4.012250831723393], [0.7927885188347537], [0.19753938953159], [7.856451936632627], [6.479942986207579], [2.656808261816911], [4.381105648436395], [5.15865688352708], [4.086452275666379], [7.84599650074806], [7.394247739851149], [4.548298642551581], [2.684068645380041], [2.805126058321613],

, -0.003357694335357264, 0.5172244776489512, -0.916124135425953, -0.6834564444641478, 0.7110524305149521, 1.4646535848302729, -1.2047415745211167, 0.5754411814225855, 0.0791196692805568, 0.9808255959579227, -0.6086329673068269, -0.3754309468889093, 0.7970132418373522, 0.38837781161060425, 0.21294292681991617, -0.6867170237869153, -1.1321009182209116, 0.9767596698390737, 0.925843221612833, 0.25092009523355296, -0.2701568121112179, 1.3096110797728748, 0.22349093619089921, -1.1522134369568184, 0.43474995142762196, 1.0479855098095472, 0.4233335654766264, 0.9333637277367053, 0.7979530640501664, -0.6285907525607816, 0.19506187146142856, 0.5341436138755178, 0.5884624987379046, -0.8496680726795944, -1.0641331878714517, -1.1861609409583544, 1.2645267727607057, 0.7669905753492215, 0.9199724013240893, 0.8408957279278326, 0.8616231120632447, 0.25749497848968383, 1.0227621137069265, 1.0814828166494985, 1.1589258483327607, 1.2840126566893832, -0.5074434913695546, 0.5373184784734018, 0.71520478649193

In [65]:
file = "data_regression.csv"
csv_file = open(file, "w")

# Write the header
write(csv_file, "xs,y\n")

# Write the data
for (xs, y) in zip(x, y)
    write(csv_file, "$xs,$y\n")
end

# Close the CSV file
close(csv_file)

In [66]:

n_hidden = 10
D = size(X, 1)
nn = Chain(
    Dense(D, n_hidden, tanh),
    Dense(n_hidden, 1)
)  
loss(x, y) = Flux.Losses.mse(nn(x), y)

loss (generic function with 1 method)

In [67]:
using Flux.Optimise: update!, Adam
opt = Adam(1e-3)
epochs = 1000
avg_loss(data) = mean(map(d -> loss(d[1], d[2]), data))
show_every = epochs/10

for epoch = 1:epochs
  for d in data
    gs = gradient(Flux.params(nn)) do
      l = loss(d...)
    end
    update!(opt, Flux.params(nn), gs)
  end
  if epoch % show_every == 0
    println("Epoch " * string(epoch))
    @show avg_loss(data)
  end
end

Epoch 100
avg_loss(data) = 0.09981518714370786


Epoch 200
avg_loss(data) = 0.0963484988101243


Epoch 300
avg_loss(data) = 0.09609773498002397


Epoch 400
avg_loss(data) = 0.09589332678381014


Epoch 500
avg_loss(data) = 0.09568133304781383


Epoch 600
avg_loss(data) = 0.09546284651383613


Epoch 700
avg_loss(data) = 0.09523957453385157


Epoch 800
avg_loss(data) = 0.09501354423591342


Epoch 900
avg_loss(data) = 0.09479017994799153


Epoch 1000
avg_loss(data) = 0.09457923415083135


In [68]:
serialize_json_nn(nn::Chain)::String = JSON.json([Dict(:weight => nn.layers[i].weight, :bias => nn.layers[i].bias) for i in range(1, length(nn.layers))])
# Export as JSON
write("nn_regression.json", serialize_json_nn(nn))
serialize("nn-binary_regression.jlb", nn)

Flux.params(nn)

Params([Float32[0.25581408; 0.24827705; … ; 0.7512668; -0.6878244;;], Float32[-1.1201183, -1.0552441, -1.1727674, 1.0405303, 1.1344997, -1.0677602, 1.9962846, 1.04113, -2.8023236, 1.1533123], Float32[1.339898 0.9142657 … -1.8449564 0.4374265], Float32[0.5303483]])

In [69]:
la = Laplace(nn; likelihood=:regression, hessian_structure=:full, subset_of_weights=:all, backend=:GGN)
fit!(la, data)
@show la.H

la.H = [7001.197956009186 4753.7221735813655 4519.258749432513 17.63783747603884 -18.094258973878368 4524.514396683313 1492.1829841875879 -4020.1341798359063 -3883.3226234233225 239.6365107005695 1414.6352260932326 965.6101898206398 905.7307687681168 7.3783590983075555 -14.295524641680856 917.3875807914883 433.10581315355375 -816.8407302396372 -943.5035808610264 84.86573711852543 204.56596151785925 233.78915687091649 163.59784186072648 1263.2477681338787 -1241.810248798225 224.46286816336215 -937.5323584526777 -236.39196933992207 618.7819112464786 -1118.7567222965881 1271.6080097071826; 4753.7221735813655 3228.4027601996786 3067.596677125257 12.360592645825818 -13.159002507555215 3072.5322862952016 1032.1842626530852 -2730.2472903114394 -2658.104678916796 167.42951007097145 965.610188796185 659.3710863506421 617.9396307291463 5.269439276191406 -10.478123999545403 626.3722100043669 301.41607032273896 -557.8096148250625 -648.0665113133145 59.911620860453695 133.47453935816884 153.5696433

31×31 Matrix{Float64}:
  7001.2      4753.72     4519.26     17.6378    …  -1118.76      1271.61
  4753.72     3228.4      3067.6      12.3606        -759.105      868.287
  4519.26     3067.6      2918.75     10.9387        -722.634      815.704
    17.6378     12.3606     10.9387    0.433763        -0.640256     8.20697
   -18.0943    -13.159     -10.8458   -1.34979         -6.2735     -19.6302
  4524.51     3072.53     2920.03     11.6594    …   -722.636      825.193
  1492.18     1032.18      935.047    11.6199        -250.98       368.494
 -4020.13    -2730.25    -2594.21    -10.4921         641.887     -734.889
 -3883.32    -2658.1     -2462.59    -14.5387         649.352     -752.363
   239.637     167.43      148.75      3.68291        -32.0289      81.5373
     ⋮                                           ⋱                   ⋮
   163.598     105.465     113.973    -4.34183        -52.6993     -31.5613
  1263.25      862.312     810.609     7.72244       -175.478      291.617
 -